In [101]:
from pathlib import Path

import tensorflow as tf
tf_session = tf.Session()
from keras import backend as K
K.set_session(tf_session)

from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

import numpy as np
import pandas as pd


# Load Input

In [171]:
root_path = Path('../..')
input_path = root_path / 'input'
poem_path = input_path / 'poems'

In [179]:
df = pd.read_csv(str(poem_path / 'haikus.csv'))
df

,0,1,2,source,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,tempslibres,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,tempslibres,"2,3",5,5
2,spring moonset --,a rice ball for,breakfast,tempslibres,"3,4",4,2
3,sunny afternoon,an old man lingers,near the mailbox,tempslibres,5,5,4
4,cinco de mayo,horses roll,in the shallows,tempslibres,5,3,4
5,quitting time,the smell of rain,in the lobby,tempslibres,3,4,4
6,waves,slowly cresting towards shore,a faint moon,tempslibres,1,"6,7",3
7,overnight rain --,the scent of orange blossoms,in a desert town,tempslibres,4,7,5
8,misty summer rain,calling pheasant,in Zen temple,tempslibres,5,4,4
9,day is done,poppies amidst,the dying grass,tempslibres,3,4,4


# Format Input for Training

In [180]:
# Duplicate lines with ambiguous syllable counts
# (syllable counts where there is a comma because
# multiple pronounciations are acceptable)

lines = set([0, 1, 2])

for i in range(3):
    lines.remove(i)
    df = df[[
        '0', '1', '2',
        #'1_syllables', '2_syllables'
    ] + ['%s_syllables' % j for j in lines]].join(
        df['%s_syllables' % i].str.split(
            ',', expand=True
        ).stack(-1).reset_index(
            level=1, drop=True
        ).rename('%s_syllables' % i)
    ).drop_duplicates()
    lines.add(i)

df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [181]:
# Drop samples that are longer that the 99th percentile of length

max_line_len = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_len) & 
    (df['1'].str.len() <= max_line_len) & 
    (df['2'].str.len() <= max_line_len)
].copy()
df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [182]:
# Pad the lines to the max line length with new lines
for i in range(3):
    # For input, duplicate the first character
    # TODO - Why?
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_len+2, 'right', '\n')
    
    # 
    #df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    
    # TODO - trying to add the next line's first character before the line breaks
    if i == 2: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_len+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_len+2, 'right', '\n')
    
max_line_len += 2

df

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
0,Memorial Day --,a shadow for each,white cross,5,5,2,MMemorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Memorial Day --\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a shadow for each\nw\n\n\n\n\n\n\n\n\n\n\n\n\n...,wwhite cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,white cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,3,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,4,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4,ssunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sunny afternoon\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aan old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\...,an old man lingers\nn\n\n\n\n\n\n\n\n\n\n\n\n\...,nnear the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,near the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,cinco de mayo,horses roll,in the shallows,5,3,4,ccinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,cinco de mayo\nh\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,hhorses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,horses roll\ni\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,iin the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
5,quitting time,the smell of rain,in the lobby,3,4,4,qquitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,quitting time\nt\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,tthe smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,the smell of rain\ni\n\n\n\n\n\n\n\n\n\n\n\n\n...,iin the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,6,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\ns\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\na\n\n\n\n\n\n\n...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,7,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\ns\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\na\n\n\n\n\n\n\n...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [184]:
df['0_out'].str.len(), max_line_len

(0        72
 1        72
 1        72
 2        72
 2        72
 3        72
 4        72
 5        72
 6        72
 6        72
 7        72
 8        72
 9        72
 10       72
 11       72
 12       72
 13       72
 13       72
 14       72
 15       72
 16       72
 17       72
 18       72
 18       72
 19       72
 20       72
 20       72
 21       72
 22       72
 23       72
          ..
 25858    72
 25859    72
 25860    72
 25860    72
 25861    72
 25862    72
 25863    72
 25864    72
 25865    72
 25866    72
 25867    72
 25868    72
 25869    72
 25870    72
 25871    72
 25872    72
 25873    72
 25874    72
 25875    72
 25876    72
 25877    72
 25878    72
 25879    72
 25880    72
 25881    72
 25882    72
 25883    72
 25884    72
 25884    72
 25885    72
 Name: 0_out, Length: 30484, dtype: int64, 72)

In [185]:
inputs = df[['0_in', '1_in', '2_in']].values

t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(inputs.flatten())
n_tokens = len(t.word_counts) + 1

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
    t.texts_to_sequences(inputs[:,i]) for i in range(3)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    t.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

# Training Model

In [186]:
LATENT_DIM = 512

In [187]:
class TrainingLine:
    def __init__(self, name, previous_line, latent_dim=LATENT_DIM):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)
        
        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(latent_dim, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)
        
        self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)
        
        if previous_line:
            initial_state = [
                Add(name='add_h_%s' % name)([
                    previous_line.lstm_h,
                    self.syllable_dense_output
                ]),
                Add(name='add_c_%s' % name)([
                    previous_line.lstm_c,
                    self.syllable_dense_output
                ])
            ]
        else:
            initial_state = [self.syllable_dense_output, self.syllable_dense_output]
            
        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)
        
        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

In [188]:
lines = []
inputs = []
outputs = []
for i in range(3):
    previous_line = lines[-1] if lines else None
    lines.append(TrainingLine('line_%s' % i, previous_line))
    inputs += [lines[-1].char_input, lines[-1].syllable_input]
    outputs.append(lines[-1].output)
    
training_model = Model(inputs, outputs)
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
training_model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
syllable_input_line_0 (InputLayer)                                (None, 1)                                   0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
char_input_line_0 (InputLayer)                                    (None, None, 74)                            0                                                                                     

In [190]:
training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=1000)

Epoch 1/1000
30484/30484 [==============================] - 509s 17ms/step - loss: 2.0921 - output_line_0_loss: 0.6900 - output_line_1_loss: 0.7635 - output_line_2_loss: 0.6386
Epoch 2/1000
30484/30484 [==============================] - 627s 21ms/step - loss: 1.8173 - output_line_0_loss: 0.5903 - output_line_1_loss: 0.6745 - output_line_2_loss: 0.5525
Epoch 3/1000
30484/30484 [==============================] - 644s 21ms/step - loss: 1.6682 - output_line_0_loss: 0.5401 - output_line_1_loss: 0.6221 - output_line_2_loss: 0.5060
Epoch 4/1000
30484/30484 [==============================] - 650s 21ms/step - loss: 1.5620 - output_line_0_loss: 0.5054 - output_line_1_loss: 0.5840 - output_line_2_loss: 0.4727
Epoch 5/1000
30484/30484 [==============================] - 617s 20ms/step - loss: 1.4772 - output_line_0_loss: 0.4777 - output_line_1_loss: 0.5540 - output_line_2_loss: 0.4455
Epoch 6/1000
30484/30484 [==============================] - 591s 19ms/step - loss: 1.4029 - output_line_0_loss: 0.4

30484/30484 [==============================] - 580s 19ms/step - loss: 0.5351 - output_line_0_loss: 0.1794 - output_line_1_loss: 0.2119 - output_line_2_loss: 0.1437
Epoch 48/1000
30484/30484 [==============================] - 572s 19ms/step - loss: 0.5311 - output_line_0_loss: 0.1782 - output_line_1_loss: 0.2105 - output_line_2_loss: 0.1424
Epoch 49/1000
30484/30484 [==============================] - 568s 19ms/step - loss: 0.5270 - output_line_0_loss: 0.1774 - output_line_1_loss: 0.2089 - output_line_2_loss: 0.1407
Epoch 50/1000
30484/30484 [==============================] - 578s 19ms/step - loss: 0.5237 - output_line_0_loss: 0.1763 - output_line_1_loss: 0.2075 - output_line_2_loss: 0.1398
Epoch 51/1000
30484/30484 [==============================] - 567s 19ms/step - loss: 0.5197 - output_line_0_loss: 0.1756 - output_line_1_loss: 0.2061 - output_line_2_loss: 0.1380
Epoch 52/1000
30484/30484 [==============================] - 572s 19ms/step - loss: 0.5161 - output_line_0_loss: 0.1747 - ou

KeyboardInterrupt: 

In [215]:
training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=1000)

Epoch 1/1000
30484/30484 [==============================] - 472s 15ms/step - loss: 0.4762 - output_line_0_loss: 0.1642 - output_line_1_loss: 0.1888 - output_line_2_loss: 0.1232
Epoch 2/1000
30484/30484 [==============================] - 571s 19ms/step - loss: 0.4717 - output_line_0_loss: 0.1629 - output_line_1_loss: 0.1872 - output_line_2_loss: 0.1217
Epoch 3/1000
30484/30484 [==============================] - 611s 20ms/step - loss: 0.4682 - output_line_0_loss: 0.1621 - output_line_1_loss: 0.1857 - output_line_2_loss: 0.1204
Epoch 4/1000
30484/30484 [==============================] - 592s 19ms/step - loss: 0.4659 - output_line_0_loss: 0.1612 - output_line_1_loss: 0.1849 - output_line_2_loss: 0.1199
Epoch 5/1000
30484/30484 [==============================] - 583s 19ms/step - loss: 0.4639 - output_line_0_loss: 0.1608 - output_line_1_loss: 0.1841 - output_line_2_loss: 0.1191
Epoch 6/1000
30484/30484 [==============================] - 571s 19ms/step - loss: 0.4620 - output_line_0_loss: 0.1

30484/30484 [==============================] - 618s 20ms/step - loss: 0.3939 - output_line_0_loss: 0.1448 - output_line_1_loss: 0.1550 - output_line_2_loss: 0.0941
Epoch 48/1000
30484/30484 [==============================] - 627s 21ms/step - loss: 0.3926 - output_line_0_loss: 0.1445 - output_line_1_loss: 0.1544 - output_line_2_loss: 0.0936
Epoch 49/1000
30484/30484 [==============================] - 615s 20ms/step - loss: 0.3904 - output_line_0_loss: 0.1439 - output_line_1_loss: 0.1533 - output_line_2_loss: 0.0932
Epoch 50/1000
30484/30484 [==============================] - 598s 20ms/step - loss: 0.3907 - output_line_0_loss: 0.1441 - output_line_1_loss: 0.1534 - output_line_2_loss: 0.0932
Epoch 51/1000
30484/30484 [==============================] - 583s 19ms/step - loss: 0.3887 - output_line_0_loss: 0.1435 - output_line_1_loss: 0.1528 - output_line_2_loss: 0.0924
Epoch 52/1000
30484/30484 [==============================] - 598s 20ms/step - loss: 0.3871 - output_line_0_loss: 0.1431 - ou

30484/30484 [==============================] - 569s 19ms/step - loss: 0.3488 - output_line_0_loss: 0.1355 - output_line_1_loss: 0.1349 - output_line_2_loss: 0.0783
Epoch 94/1000
30484/30484 [==============================] - 576s 19ms/step - loss: 0.3480 - output_line_0_loss: 0.1357 - output_line_1_loss: 0.1341 - output_line_2_loss: 0.0783
Epoch 95/1000
30484/30484 [==============================] - 592s 19ms/step - loss: 0.3466 - output_line_0_loss: 0.1355 - output_line_1_loss: 0.1336 - output_line_2_loss: 0.0774
Epoch 96/1000
30484/30484 [==============================] - 571s 19ms/step - loss: 0.3460 - output_line_0_loss: 0.1353 - output_line_1_loss: 0.1337 - output_line_2_loss: 0.0770
Epoch 97/1000
30484/30484 [==============================] - 583s 19ms/step - loss: 0.3453 - output_line_0_loss: 0.1351 - output_line_1_loss: 0.1327 - output_line_2_loss: 0.0775
Epoch 98/1000
30484/30484 [==============================] - 586s 19ms/step - loss: 0.3439 - output_line_0_loss: 0.1347 - ou

30484/30484 [==============================] - 611s 20ms/step - loss: 0.3174 - output_line_0_loss: 0.1303 - output_line_1_loss: 0.1197 - output_line_2_loss: 0.0673
Epoch 139/1000
30484/30484 [==============================] - 626s 21ms/step - loss: 0.3175 - output_line_0_loss: 0.1302 - output_line_1_loss: 0.1198 - output_line_2_loss: 0.0675
Epoch 140/1000
30484/30484 [==============================] - 626s 21ms/step - loss: 0.3166 - output_line_0_loss: 0.1297 - output_line_1_loss: 0.1195 - output_line_2_loss: 0.0673
Epoch 141/1000
30484/30484 [==============================] - 620s 20ms/step - loss: 0.3162 - output_line_0_loss: 0.1301 - output_line_1_loss: 0.1192 - output_line_2_loss: 0.0669
Epoch 142/1000
30484/30484 [==============================] - 602s 20ms/step - loss: 0.3140 - output_line_0_loss: 0.1296 - output_line_1_loss: 0.1182 - output_line_2_loss: 0.0662
Epoch 143/1000
30484/30484 [==============================] - 642s 21ms/step - loss: 0.3140 - output_line_0_loss: 0.1296

KeyboardInterrupt: 

In [216]:
training_model.save('new_model_3.hdf5')

/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_line_0 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'syllable_dense_line_0_5/Relu:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'syllable_dense_line_0_5/Relu:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_line_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'add_h_line_1_4/add:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'add_c_line_1_4/add:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages

In [30]:
class HCInput:
    def __init__(self, name, latent_dim=LATENT_DIM):
        self.lstm_h = Input(shape=(latent_dim,), name='h_in_%s' % name)
        self.lstm_c = Input(shape=(latent_dim,), name='c_in_%s' % name)
        
name = 'generator_line_1'

l1 = Line(name, None)


In [108]:
name = 'generator_line_2'
p2 = HCInput(name)
l2 = Line(name, p2)

l2.syllable_dense.set_weights(lines[1].syllable_dense.get_weights())
l2.lstm.set_weights(lines[1].lstm.get_weights())
l2.output_dense.set_weights(lines[1].output_dense.get_weights())

In [109]:
print(l2.output.eval(feed_dict={
    l2.char_input : [[X[0][0][0]]],
    l2.syllable_input: [[X_syllables[0][0]]],
    p2.lstm_c: np.zeros((1, LATENT_DIM)),
    p2.lstm_h: np.zeros((1, LATENT_DIM)),
}, session=tf_session))

[[[3.9216536e-07 3.5172909e-06 2.3977848e-04 5.1234281e-01 7.6161664e-06
   1.5869097e-01 4.3124170e-03 1.2343503e-05 5.4193346e-04 7.9986408e-02
   2.7995504e-04 5.5539590e-07 3.3351305e-05 1.7019777e-07 3.4399509e-02
   1.3412108e-01 1.3112953e-05 5.3419091e-05 1.5718451e-06 2.2147278e-06
   3.8094260e-02 5.0196954e-04 2.8779911e-02 3.6528202e-06 4.2539563e-05
   7.0325565e-04 5.6407521e-03 1.0337839e-04 6.7814726e-05 1.5159931e-04
   1.5373316e-05 1.3540406e-05 8.8866444e-05 1.5705133e-04 5.1120578e-06
   1.1866817e-05 2.1660942e-04 2.2514409e-05 2.6205216e-05 5.8846103e-06
   1.5222559e-04 4.3577688e-07 2.6185131e-05 1.0960441e-05 3.0419505e-06
   4.3921611e-07 6.5289564e-06 3.1762472e-06 5.1846808e-05 1.2357714e-06
   2.1658874e-05 1.0838974e-06 2.2545983e-06 3.1417467e-06 7.5716810e-07
   2.3695874e-07 6.7183043e-07 2.8423758e-06 3.1195991e-06 9.5575274e-07
   2.1149167e-06 8.1350197e-07 4.5901362e-07 2.4374137e-06 2.5152687e-07
   4.3921526e-07 4.4280486e-07 1.1952369e-06 1.5179

In [83]:
l1.syllable_dense.set_weights(lines[0].syllable_dense.get_weights())
l1.lstm.set_weights(lines[0].lstm.get_weights())
l1.output_dense.set_weights(lines[0].output_dense.get_weights())

In [105]:

#init = tf.initialize_all_variables()
#with tf.Session() as s:
    #s.run(init)
    #l1.output.eval()
print(l1.output.eval(feed_dict={
    l1.char_input : [[X[0][0][0]]],
    l1.syllable_input: [[X_syllables[0][0]]]
}, session=tf_session))

FailedPreconditionError: Attempting to use uninitialized value output_generator_line_1/kernel
	 [[node output_generator_line_1/kernel/read (defined at /home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](output_generator_line_1/kernel)]]

Caused by op 'output_generator_line_1/kernel/read', defined at:
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/d

In [92]:
tf_session.run(
    [l1.output, l1.lstm_c],
    feed_dict={
        l1.char_input : [[X[0][0][0]]],
        l1.syllable_input: [[X_syllables[0][0]]]
    }
)

[array([[[1.76697860e-15, 5.92968860e-12, 5.10904741e-13, 1.04402630e-12,
          7.65024491e-13, 2.97301420e-12, 5.16687144e-11, 2.66838906e-13,
          1.91477684e-11, 7.47263570e-11, 3.79553431e-13, 1.05872012e-14,
          7.22546463e-15, 1.04571889e-11, 2.29380438e-11, 1.00000000e+00,
          6.34109345e-12, 1.40137977e-12, 4.05155916e-14, 4.29198170e-12,
          3.17661980e-10, 4.79395279e-09, 1.50053259e-09, 1.67381317e-11,
          2.94760592e-12, 2.23507688e-11, 1.29700417e-12, 2.69770067e-10,
          2.44515491e-10, 1.35983669e-09, 5.86872981e-11, 9.89551843e-11,
          2.06421234e-11, 1.82141624e-09, 4.87898055e-11, 5.45356742e-15,
          2.73636279e-12, 7.22586505e-11, 4.29104842e-12, 1.74455936e-10,
          6.02291828e-12, 2.32974438e-11, 4.58175652e-13, 1.34999108e-12,
          1.05733339e-11, 4.13906062e-12, 2.31504694e-11, 1.84662051e-12,
          2.43626767e-14, 2.70372222e-14, 2.42930524e-12, 9.59201121e-12,
          1.47016926e-12, 5.24562838e-

In [93]:
l1.lstm.states

[None, None]

In [87]:
l1.lstm_c.eval(session=tf_session)

InvalidArgumentError: You must feed a value for placeholder tensor 'syllable_input_generator_line_1' with dtype float and shape [?,1]
	 [[node syllable_input_generator_line_1 (defined at /home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517)  = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'syllable_input_generator_line_1', defined at:
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.s

In [77]:
lines[0].syllable_dense.get_weights()

[array([[ 5.46481013e-02, -2.57733073e-02,  4.52999063e-02,
          1.79725230e-01, -5.55427074e-02, -7.81552792e-02,
         -3.34872603e-02, -7.38996081e-04, -1.23093426e-02,
          4.98037115e-02, -2.40507387e-02,  1.90334190e-02,
         -2.29553655e-02, -4.28513512e-02, -9.42896307e-02,
         -1.33944559e-04,  5.88149577e-02, -1.22766495e-02,
         -2.78256200e-02,  4.80774045e-02, -1.86152831e-02,
         -7.27259275e-03,  3.10732573e-02, -2.03387886e-02,
          3.96197103e-02, -9.98452678e-02, -2.14310735e-03,
         -2.14907364e-03, -2.19272822e-03, -6.82606921e-02,
         -4.02438268e-02, -2.33108774e-02, -2.07900628e-02,
         -1.79877505e-01, -1.04780711e-01, -5.46084978e-02,
          1.20561160e-01, -4.78963554e-03, -6.69527277e-02,
          1.75244939e-02, -2.14169826e-03, -8.05112869e-02,
         -8.27796608e-02, -2.67894492e-02, -1.81851685e-02,
          1.02168746e-01, -8.87685716e-02, -9.48045999e-02,
         -9.20971632e-02,  7.17438757e-0

In [74]:
K.get_session().run(init)

In [72]:
K.set_session()

TypeError: set_session() missing 1 required positional argument: 'session'

In [107]:
training_model.load_weights('new_model.hdf5')

In [80]:
training_model.to_json()

'{"class_name": "Model", "config": {"name": "model_2", "layers": [{"name": "syllable_input_line_0", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 1], "dtype": "float32", "sparse": false, "name": "syllable_input_line_0"}, "inbound_nodes": []}, {"name": "char_input_line_0", "class_name": "InputLayer", "config": {"batch_input_shape": [null, null, 74], "dtype": "float32", "sparse": false, "name": "char_input_line_0"}, "inbound_nodes": []}, {"name": "syllable_dense_line_0", "class_name": "Dense", "config": {"name": "syllable_dense_line_0", "trainable": true, "units": 512, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["sylla

In [110]:
writer = tf.summary.FileWriter('logs', tf_session.graph)
print(l2.output.eval(feed_dict={
    l2.char_input : [[X[0][0][0]]],
    l2.syllable_input: [[X_syllables[0][0]]],
    p2.lstm_c: np.zeros((1, LATENT_DIM)),
    p2.lstm_h: np.zeros((1, LATENT_DIM)),
}, session=tf_session))
#writer.close()

[[[3.9216536e-07 3.5172909e-06 2.3977848e-04 5.1234281e-01 7.6161664e-06
   1.5869097e-01 4.3124170e-03 1.2343503e-05 5.4193346e-04 7.9986408e-02
   2.7995504e-04 5.5539590e-07 3.3351305e-05 1.7019777e-07 3.4399509e-02
   1.3412108e-01 1.3112953e-05 5.3419091e-05 1.5718451e-06 2.2147278e-06
   3.8094260e-02 5.0196954e-04 2.8779911e-02 3.6528202e-06 4.2539563e-05
   7.0325565e-04 5.6407521e-03 1.0337839e-04 6.7814726e-05 1.5159931e-04
   1.5373316e-05 1.3540406e-05 8.8866444e-05 1.5705133e-04 5.1120578e-06
   1.1866817e-05 2.1660942e-04 2.2514409e-05 2.6205216e-05 5.8846103e-06
   1.5222559e-04 4.3577688e-07 2.6185131e-05 1.0960441e-05 3.0419505e-06
   4.3921611e-07 6.5289564e-06 3.1762472e-06 5.1846808e-05 1.2357714e-06
   2.1658874e-05 1.0838974e-06 2.2545983e-06 3.1417467e-06 7.5716810e-07
   2.3695874e-07 6.7183043e-07 2.8423758e-06 3.1195991e-06 9.5575274e-07
   2.1149167e-06 8.1350197e-07 4.5901362e-07 2.4374137e-06 2.5152687e-07
   4.3921526e-07 4.4280486e-07 1.1952369e-06 1.5179

In [111]:
writer.close()

In [192]:
class GeneratorLine:
    def __init__(self, name, training_line, latent_dim=LATENT_DIM):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)
        
        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(latent_dim, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)
        
        self.h_input = Input(shape=(latent_dim,), name='h_input_%s' % name)
        self.c_input = Input(shape=(latent_dim,), name='c_input_%s' % name)
        initial_state = [self.h_input, self.c_input]
        
        self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)
            
        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)
        
        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)
        
        self.syllable_dense.set_weights(training_line.syllable_dense.get_weights())
        self.lstm.set_weights(training_line.lstm.get_weights())
        self.output_dense.set_weights(training_line.output_dense.get_weights())
        

In [217]:
generator_lines = [GeneratorLine('generator_line_%s' % i, lines[i]) for i in range(3)]
#l0 = GeneratorLine('training_line_0', lines[0])

In [116]:
syllable_dense_out = tf_session.run(
    l0.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l0.syllable_input: [[X_syllables[0][0]]]
    }
)
syllable_dense_out.shape

(1, 512)

In [119]:
char, h, c = tf_session.run(
    [l0.output, l0.lstm_h, l0.lstm_c],
    feed_dict={
        l0.char_input : [[X[0][0][0]]],
        l0.h_input: syllable_dense_out,
        l0.c_input: syllable_dense_out
    }
)

In [123]:
t.sequences_to_texts([[np.argmax(char)]])

['m']

In [124]:
np_utils.to_categorical(15, num_classes=n_tokens)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

In [126]:
char, h, c = tf_session.run(
    [l0.output, l0.lstm_h, l0.lstm_c],
    feed_dict={
        l0.char_input: [[np_utils.to_categorical(15, num_classes=n_tokens)]],
        l0.h_input: h,
        l0.c_input: c
    }
)

In [129]:
t.sequences_to_texts([[np.argmax(char)]])

['o']

In [197]:
h = tf_session.run(
    l0.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l0.syllable_input: [[5]]
    }
)
c = h

line = [5]

end = False
next_char = None
for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l0.output, l0.lstm_h, l0.lstm_c],
        feed_dict={
            l0.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l0.h_input: h,
            l0.c_input: c
        }
    )
    
    char = np.argmax(char)
    if char == 1 and not end:
        end = True
    if char != 1 and end:
        next_char = char
        char = 1
        
    line.append(char)

In [198]:
line

[5,
 5,
 2,
 7,
 4,
 10,
 3,
 5,
 15,
 2,
 6,
 17,
 2,
 7,
 14,
 15,
 15,
 3,
 10,
 2,
 25,
 25,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [199]:
t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' ')

'a stream of summer --'

In [200]:
next_char

4

In [201]:
l1 = GeneratorLine('training_line_1', lines[1])

In [204]:
h = tf_session.run(
    l1.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l1.syllable_input: [[7]]
    }
)
h = s + h
c = h

line = [next_char]

end = False
next_char = None
for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l1.output, l1.lstm_h, l1.lstm_c],
        feed_dict={
            l1.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l1.h_input: h,
            l1.c_input: c
        }
    )
    
    char = np.argmax(char)
    if char == 1 and not end:
        end = True
    if char != 1 and end:
        next_char = char
        char = 1
        
    line.append(char)

In [205]:
t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' ')

'the down shatter you could be sent'

In [159]:
s = tf_session.run(
    l0.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l0.syllable_input: [[7]]
    }
)
h = s + h
c = h

line = [4]

for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l0.output, l0.lstm_h, l0.lstm_c],
        feed_dict={
            l0.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l0.h_input: h,
            l0.c_input: c
        }
    )
    
    line.append(np.argmax(char))

In [160]:
t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n')#.replace(' ', '').replace('\n', ' ')

' t e m p l e\na n d\nr u n\nt r a p t a m e d b r o o k s d a - b a t w e d f e l i c t a l m w a l e f u l e e d b a r b a b s e w a r d s'

In [221]:
l0 = generator_lines[0]
l1 = generator_lines[1]
l2 = generator_lines[2]

h = tf_session.run(
    l0.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l0.syllable_input: [[5]]
    }
)
c = h

line = [6]

end = False
next_char = None
for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l0.output, l0.lstm_h, l0.lstm_c],
        feed_dict={
            l0.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l0.h_input: h,
            l0.c_input: c
        }
    )
    
    char = np.argmax(char)
    if char == 1 and not end:
        end = True
    if char != 1 and end:
        next_char = char
        char = 1
        
    line.append(char)
    
print(t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' '))


h = tf_session.run(
    l1.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l1.syllable_input: [[7]]
    }
)
h = s + h
c = h

line = [next_char]

end = False
next_char = None
for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l1.output, l1.lstm_h, l1.lstm_c],
        feed_dict={
            l1.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l1.h_input: h,
            l1.c_input: c
        }
    )
    
    char = np.argmax(char)
    if char == 1 and not end:
        end = True
    if char != 1 and end:
        next_char = char
        char = 1
        
    line.append(char)
    
print(t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' '))
    
h = tf_session.run(
    l2.syllable_dense_output,
    feed_dict={
        #l0.char_input : [[X[0][0][0]]],
        l2.syllable_input: [[5]]
    }
)
h = s + h
c = h

line = [next_char]

end = False
next_char = None
for i in range(max_line_len):
    char, h, c = tf_session.run(
        [l2.output, l2.lstm_h, l2.lstm_c],
        feed_dict={
            l2.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
            l2.h_input: h,
            l2.c_input: c
        }
    )
    
    char = np.argmax(char)
    if char == 1 and not end:
        end = True
    if char != 1 and end:
        next_char = char
        char = 1
        
    line.append(char)
    
print(t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' '))


one of the giunta
x the other tastes are one look then long
the lives of voice of highway
